In [10]:
%run ./dataset.ipynb # runs file in context of current kernel, workaround for local import

In [3]:
import numpy as np
from torch import optim, nn, cuda
import tqdm

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = 64 // 4
#       self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(100, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 3, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(3, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = 64 // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 2), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

In [6]:
# Loss function
adversarial_loss = nn.CrossEntropyLoss()

gen = Generator()
disc = Discriminator()

optimizer_g = optim.Adam(gen.parameters(), lr=0.01, betas=(0.9,0.999))
optimizer_d = optim.Adam(disc.parameters(), lr=0.01, betas=(0.9,0.999))

image_iterator = GenerateIterator('./images_cropped')

In [11]:
if cuda.is_available():
    adversarial_loss = adversarial_loss.cuda()
    gen = gen.cuda()
    disc = disc.cuda()
    
start_epoch = 1
losses = [[],[]]

for epoch in range(start_epoch,101):
    progress_bar = tqdm.tqdm(image_iterator)
    for images in progress_bar:
        valid_image_labels = torch.ones(images.shape[0]).type(torch.LongTensor)
        fake_image_labels = torch.zeros(images.shape[0]).type(torch.LongTensor)
        seed = torch.from_numpy(np.random.normal(0,1, size=(images.shape[0], 100))).type(torch.FloatTensor)
        if cuda.is_available():
            valid_image_labels, fake_image_labels, images, seed = valid_image_labels.cuda(), fake_image_labels.cuda(), images.cuda(), seed.cuda()
            
        # --- train generator ---
        fake_images = gen(seed)
        gen_loss = adversarial_loss(disc(fake_images), valid_image_labels)
        losses[0].append(gen_loss)
        
        optimizer_g.zero_grad()
        gen_loss.backward(retain_graph=True)
        optimizer_g.step()
        
        # --- train discriminator ---
        real_loss = adversarial_loss(disc(images), valid_image_labels)
        fake_loss = adversarial_loss(disc(fake_images), fake_image_labels)
        # divide by 2 or else loss is weighted 2x towards disc vs gen
        disc_loss = (real_loss + fake_loss) / 2
        losses[1].append(disc_loss)
        
        optimizer_d.zero_grad()
        disc_loss.backward(retain_graph=True)
        optimizer_d.step()
        
        progress_bar.set_description(f"disc loss: {disc_loss} || gen loss: {gen_loss}")
        


  0%|                                                                                         | 0/1287 [00:00<?, ?it/s]

PicklingError: Can't pickle <class '__main__.Dataset'>: it's not the same object as __main__.Dataset

In [ ]:
# plot losses